In [45]:
# Importando as bibliotecas
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

import requests

from bs4 import BeautifulSoup

import pandas as pd

import time
from datetime import date

In [46]:
# Criando a instância no WebDriver
srv = Service("WebDriver\\gchromedriver.exe") 
opt = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=srv, options=opt)

In [47]:
with open('login.txt', 'r') as file:
    linhas = file.readlines()

credentials = {"login": linhas[0].strip(), "password": linhas[1].strip()}

In [48]:
def login_linkedin(user, password):
    # Abrindo a página de Login do LinkedIn
    driver.get('https://www.linkedin.com/login/')
    time.sleep(1) # aguardar 1 segundos para a página abrir

    # Inserindo o usuário
    usuario = driver.find_element('id','username') # acessando o campo usuário
    usuario.send_keys(credentials['login']) # enviando a string com o usuário

    # Inserindo a senha
    senha = driver.find_element('id','password') # acessando o campo senha
    senha.send_keys(credentials['password']) # enviando a string com a senha

    # Clicando no botão 'Entrar' para acessar a conta no LinkedIn
    driver.find_element("xpath","//button[@type='submit']").click()

login_linkedin(credentials['login'], credentials['password'])

In [59]:
def extrair_info_formacao_academica(item_educacao):
    # Extraindo o nome da Instituição de Ensino
    nome_instituicao_loc = item_educacao.find('div', {'class':'display-flex align-items-center mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
    nome_instituicao = nome_instituicao_loc.get_text().strip()

    # Separando o Tipo da Formação do Nome do Formação
    tipo_nome_formacao_loc = item_educacao.find('span',{'class':'t-14 t-normal'}).find('span',{'aria-hidden':'true'})
    tipo_nome_formacao = tipo_nome_formacao_loc.get_text().strip().split(',')

    # Tipo da Formação
    tipo_formacao = tipo_nome_formacao[0].strip()

    # Nome da Formação
    nome_formacao = tipo_nome_formacao[1].strip()

    # Extraindo a duração da Formação
    duracao_formacao_loc = item_educacao.find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
    duracao_formacao = duracao_formacao_loc.get_text().strip()
    
    # Retornando as informações
    return nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao

def extrair_info_formacao_academica_2(item_educacao):
    # Extraindo o nome da Instituição de Ensino
    nome_instituicao_loc = item_educacao.find('span', {'class':'mr1 hoverable-link-text t-bold'}).find('span',{'aria-hidden':'true'})
    nome_instituicao = nome_instituicao_loc.get_text().strip()

    # Extraindo a duração da Formação (caso esteja disponível)
    try:
        duracao_formacao_loc = item_educacao.find('span',{'class':'t-14 t-normal t-black--light'}).find('span',{'aria-hidden':'true'})
        duracao_formacao = duracao_formacao_loc.get_text().strip()
    except:
        duracao_formacao = ''
    
    # Campos não disponíveis `nome_formacao` e `tipo_formacao`
    nome_formacao = ''
    tipo_formacao = ''
    
    # Retornando as informações
    return nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao

# Função para extrair as informações referente à Formação Acadêmica mais recente
def checar_formacao_academica(perfil_id, instituiçao):
    # Acessando a página referente às formações acadêmicas
    formacao_url = 'https://www.linkedin.com/in/{}/details/education/'.format(perfil_id)
    driver.get(formacao_url)
    
    # Lendo a página do início ao fim
    inicio = time.time()
    posicao_inicial_rolamento = 0
    posicao_final_rolamento = 1000

    while True:
        driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")

        posicao_inicial_rolamento = posicao_final_rolamento
        posicao_final_rolamento += 1000

        # Aguardando 2 segundos
        time.sleep(2)

        fim = time.time()

        # Executar o script por 5 segundos
        if round(fim - inicio) > 5:
            break

    # Salvando o código fonte da página em uma variável
    src_educacao = driver.page_source

    # Utilizando o código fonte para gerar um objeto Beautiful Soup
    soup_educacao = BeautifulSoup(src_educacao,'lxml')
    
    # Obtendo o HTML de extração das formações acadêmicas
    try:
        educacao = soup_educacao.find('div',{'class':'scaffold-finite-scroll__content'}).find('ul').find_all('li')
    except:
        return None
            
    # Retornando as informações extraídas
    formacoes = []

    for edc in educacao:
        if(edc.find('div', {'class':'display-flex align-items-center mr1 hoverable-link-text t-bold'})):
            try:
                try:
                    nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = extrair_info_formacao_academica(edc)
                except:
                    nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = extrair_info_formacao_academica(edc)
                formacoes.append({'instituicao': nome_instituicao, 'tipo': tipo_formacao, 'area': nome_formacao, 'duracao': duracao_formacao})
            except:
                nome_instituicao, tipo_formacao, nome_formacao, duracao_formacao = '', '', '', ''
    for nome in instituiçao:
        for f in formacoes:
            if(nome.upper() == f['instituicao'].upper()):
                return True
    
    return False

teste = checar_formacao_academica('nicolashdx', ['UFMG', 'Universidade Federal de Minas Gerais'])

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.61)
Stacktrace:
	GetHandleVerifier [0x00007FF730607032+63090]
	(No symbol) [0x00007FF730572C82]
	(No symbol) [0x00007FF73040EC65]
	(No symbol) [0x00007FF7303ECA7C]
	(No symbol) [0x00007FF73047D687]
	(No symbol) [0x00007FF730492AC1]
	(No symbol) [0x00007FF730476D83]
	(No symbol) [0x00007FF7304483A8]
	(No symbol) [0x00007FF730449441]
	GetHandleVerifier [0x00007FF730A025AD+4238317]
	GetHandleVerifier [0x00007FF730A3F70D+4488525]
	GetHandleVerifier [0x00007FF730A379EF+4456495]
	GetHandleVerifier [0x00007FF7306E0576+953270]
	(No symbol) [0x00007FF73057E54F]
	(No symbol) [0x00007FF730579224]
	(No symbol) [0x00007FF73057935B]
	(No symbol) [0x00007FF730569B94]
	BaseThreadInitThunk [0x00007FFF13B07344+20]
	RtlUserThreadStart [0x00007FFF14F026B1+33]


In [50]:
pessoas = []

with open('nomes.txt', 'r', encoding='utf-8') as file:
    linhas = file.readlines()

for linha in linhas:
    pessoas.append({'Name':linha.replace('\n', ''), 'Public Name':'','Curso':'', 'Linkedin ID': '', 'Linkedin URL': ''})

In [51]:
API_KEY = open('api_key').read()

SEARCH_ENGINE_ID = open('search_engine_id').read()

In [57]:
def search_name(nome, params):
    query_string = f'{nome.split(' ')[0]} {nome.split(' ')[-1]} {params}'
    url = f"https://www.googleapis.com/customsearch/v1?key={API_KEY}&cx={SEARCH_ENGINE_ID}&q={query_string}"
    response = requests.get(url)
    data = response.json()
    if 'items' in data:
        return data['items']
    else:
        return []

busca = 'UFMG inurl:"linkedin"'

for p in pessoas:
    results = search_name(p['Name'], busca)
    if len(results) > 0:
        for r in results:
            if "br.linkedin.com/in/" in r['link']:
                profile_id = r['link'].split('/')[-1] if(len(r['link'].split('/')[-1])>0) else r['link'].split('/')[-2]
                if(checar_formacao_academica(profile_id, ["UFMG", "Universidade Federal de Minas Gerais"])):
                    p['Linkedin ID'] = profile_id
                    p['Linkedin URL'] = r['link']
                    break
    else:
        p['Linkedin URL'] = ''
        p['Linkedin ID'] = ''

AttributeError: 'NoneType' object has no attribute 'find'

In [53]:
df = pd.DataFrame(pessoas)

df.head(20)

,Name,Public Name,Curso,Linkedin ID,Linkedin URL
0,Carlos Ermidio Ferreira Caetano,,,ccaetano-junior,https://br.linkedin.com/in/ccaetano-junior
1,Frederico Ferreira Viana Matos,,,tom%C3%A1s-corr%C3%AAa-937866221,https://br.linkedin.com/in/tom%C3%A1s-corr%C3%...
2,Ricardo Luiz da Silva Adriano,,,ricardoadf,https://br.linkedin.com/in/ricardoadf
3,André Paim Lemos,,,andrepaimlemos,https://br.linkedin.com/in/andrepaimlemos
4,Alessandro Fernandes Moreira,,,alessandro-moreira-37b0bbaa,https://br.linkedin.com/in/alessandro-moreira-...
5,Braz de Jesus Cardoso Filho,,,braz-de-jesus-cardoso-filho-02b3a9a2,https://br.linkedin.com/in/braz-de-jesus-cardo...
6,Marcos Antônio Severo Mendes,,,marcomendes,https://br.linkedin.com/in/marcomendes
7,Adriano Vilela Barbosa,,,adriano-ferraz-46930611a,https://br.linkedin.com/in/adriano-ferraz-4693...
8,Luciano de Errico,,,pedrodoc,https://br.linkedin.com/in/pedrodoc
9,Antônio de Pádua Braga,,,apbraga,https://br.linkedin.com/in/apbraga


In [ ]:
profile_id = "nicolashdx"
url = f'https://www.linkedin.com/in/{profile_id}/'

def get_profile(perfil_url):
    # Acessando o perfil
    driver.get(perfil_url)
    time.sleep(1) # aguardar 1 segundos para a página abrir

def download_pdf():
    try:
        # Clicando no botão '...' na seção de "Introdução" do perfil
        driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/button').click()
        time.sleep(1) # aguardar 1 segundo

        # Clicando no botão "Salvar como PDF"
        driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/div/div/ul/li[2]/div').click()

    except:
        # Clicando no botão 'Mais' na seção de "Introdução" do perfil
        driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/button').click()
        time.sleep(1) # aguardar 1 segundo

        # Clicando no botão "Salvar como PDF"
        driver.find_element('xpath','/html/body/div[5]/div[3]/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[2]/div/div/ul/li[2]/div').click()

# Função para ler a página inicial
def read_profile():
    # Lendo a página do início ao fim
    inicio = time.time()
    posicao_inicial_rolamento = 0
    posicao_final_rolamento = 1000

    while True:
        driver.execute_script(f"window.scrollTo({posicao_inicial_rolamento},{posicao_final_rolamento})")

        posicao_inicial_rolamento = posicao_final_rolamento
        posicao_final_rolamento += 1000

        # Aguardando 2 segundos
        time.sleep(2)

        fim = time.time()

        # Executar o script por 10 segundos
        if round(fim - inicio) > 10:
            break
    
    # Salvando o código fonte da página em uma variável
    src = driver.page_source

    # Utilizando o código fonte para gerar um objeto Beautiful Soup
    soup = BeautifulSoup(src,'lxml')
    
    # Retornando o Código Fonte
    return soup

# Função para Extrair as informações contidas na seção de Introdução da página
def extrair_introducao(codigo_fonte_principal):
    # Obtendo o HTML de extração da introdução
    intro = codigo_fonte_principal.find('div',{'class': 'mt2 relative'})
    
    # Localizando e extraindo o nome do candidato
    nome_loc = intro.find('h1')
    nome = nome_loc.get_text().strip()

    # Localizando e extraindo o título profissional do candidato
    titulo_loc = intro.find('div', {'class': 'text-body-medium break-words'})
    titulo = titulo_loc.get_text().strip()

    # Localizando e extraindo o endereço de residência
    try:
        localizacao_loc = intro.find('span', {'class': 'text-body-small inline t-black--light break-words'})
        localizacao = localizacao_loc.get_text().strip()
    except:
        localizacao = ''

    # Localizando e extraindo a organização de trabalho
    trabalho_loc = intro.find('div', {'class': 'inline-show-more-text'})
    if trabalho_loc == None:
        trabalho = ''
    else:
        trabalho = trabalho_loc.get_text().strip()

    # Localizando e extraindo a instituição da formação acadêmica
    estudo_loc = intro.find_all('div', {'class': 'inline-show-more-text'})
    if estudo_loc == None:
        estudo = ''
    else:
        try:
            estudo = estudo_loc[1].get_text().strip()
        except:
            estudo = ''
        
    # Retornando as informações extraídas
    return nome, titulo, localizacao, trabalho, estudo


# get_profile(url)

# profile = read_profile()

# nome, titulo, localizacao, trabalho, estudo = extrair_introducao(profile)